In [1]:
import pandas as pd
import pickle as pkl

df = pd.read_csv('test.csv')
df.drop(['TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC', 'Q027', 'SG_UF_RESIDENCIA', 'TP_PRESENCA_MT'], axis=1, inplace=True)
df.head()

,NU_INSCRICAO,CO_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,IN_BAIXA_VISAO,...,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q047
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,43,19,F,1,1,1,3,1,0,...,NaN,NaN,NaN,E,E,H,B,B,C,A
1,177f281c68fa032aedbd842a745da68490926cd2,15,24,M,3,2,1,4,1,0,...,NaN,NaN,NaN,B,C,B,A,A,C,A
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,29,16,F,2,1,3,0,1,0,...,NaN,NaN,NaN,E,F,G,B,B,A,D
3,5c356d810fa57671402502cd0933e5601a2ebf1e,41,17,F,1,1,2,0,2,0,...,120.0,40.0,480.0,E,E,E,C,B,B,A
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,53,19,F,1,1,1,1,1,0,...,160.0,80.0,720.0,E,E,E,B,B,B,A


In [2]:
lista_nan = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 
       'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3',
       'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO']

for col in lista_nan:
    df[col].fillna(df[col].mean(), inplace=True)

In [3]:
lista_encoder = ['TP_SEXO', 'Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047']

for i in lista_encoder:
    with open(f'./encoder/encoder_{i}.pkl', 'rb') as f:
        le = pkl.load(f)
    
    df[i] = le.transform(df[i])

In [4]:
filename = './models/lr.sav'
lr = pkl.load(open(filename, 'rb'))

In [5]:
x = df[['CO_UF_RESIDENCIA', 'NU_IDADE', 'TP_SEXO', 'TP_COR_RACA',
        'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA',
        'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ',
        'IN_DISLEXIA', 'IN_DISCALCULIA', 'IN_SABATISTA', 'IN_GESTANTE',
        'IN_IDOSO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
        'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'TP_LINGUA',
        'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3',
        'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002',
        'Q006', 'Q024', 'Q025', 'Q026', 'Q047']].copy()

In [7]:
with open(f'./encoder/scale.pkl', 'rb') as f:
    scale = pkl.load(f)
    
x = scale.transform(x)

In [9]:
y_pred = lr.predict(x)

In [10]:
df['IN_TREINEIRO'] = y_pred

In [11]:
df[['NU_INSCRICAO', 'IN_TREINEIRO']].to_csv('answer.csv', index=False)

In [12]:
df['IN_TREINEIRO'].value_counts()

0    3968
1     602
Name: IN_TREINEIRO, dtype: int64